### Load environment variables

In [1]:
import os
import sys  
sys.path.append("..")  

from dotenv import load_dotenv
_ = load_dotenv('.env')

# This lines should be modified according to your environment variables
os.environ['OPENAI_API_TYPE'] =  os.getenv('OPENAI_API_TYPE')
os.environ['OPENAI_API_VERSION'] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ['OPENAI_API_BASE'] = os.getenv("AZURE_OPENAI_API_ENDPOINT")
os.environ['OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['LANGCHAIN_HUB_API_KEY'] = os.getenv("LANG_SMITH_API_KEY")  ## This is not necessary. You can pull without this.
LLM_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_NAME")

### Get a random persona

In [2]:
from user_persona import UserPersona, get_random_user_persona
persona = get_random_user_persona().json()
persona

'{"gender": "male", "name": "Dennis Estrada", "language": "English", "nationality": "United_Kingdom", "age": 30, "hobbies": ["painting", "photography", "astronomy"], "talkative": true, "characteristics": ["NotImpulsive", "NotOptimistic"], "education_level": "genius_level"}'

### Get OpenAI functions

In [3]:
from user_persona import UserProfile
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
openai_function_profile = convert_pydantic_to_openai_function(UserProfile)

# You can also get openai_function_profile in this way
# with open('openai_function_profile.json', 'r') as f:  
#     openai_function_profile = json.load(f)

### Fetch the prompt and create a chain (using LCEL)


In [4]:
from langchain import hub
from langchain.chat_models import AzureChatOpenAI

prompt = hub.pull("jet-taekyo-lee/persona-description")
chat_model = AzureChatOpenAI(deployment_name=LLM_DEPLOYMENT_NAME, temperature=0).bind(functions=[openai_function_profile], function_call={'name':'UserProfile'})


chain = prompt | chat_model

## Get a response

In [5]:
response = chain.invoke({'persona':persona})
print(response.additional_kwargs['function_call']['arguments'])

{
  "original_persona": {
    "gender": "male",
    "name": "Dennis Estrada",
    "language": "English",
    "nationality": "United_Kingdom",
    "age": 30,
    "hobbies": ["painting", "photography", "astronomy"],
    "talkative": true,
    "characteristics": ["NotImpulsive", "NotOptimistic"],
    "education_level": "genius_level"
  },
  "refined_persona": {
    "gender": "male",
    "name": "Dennis Estrada",
    "language": "English",
    "nationality": "United_Kingdom",
    "age": 30,
    "hobbies": ["painting", "photography", "astronomy"],
    "talkative": true,
    "characteristics": ["Patient", "Pessimistic"],
    "education_level": "genius_level"
  },
  "refined_persona_description": {
    "description_of_persona": "Dennis Estrada is a 30-year-old male from the United Kingdom. He is a genius-level intellect who communicates in English. Dennis is a talkative individual with a passion for painting, photography, and astronomy. Despite his high intelligence, he tends to be pessimisti